In [2]:
from Bio import AlignIO
from Bio import SeqIO
import pandas as pd
import os
import numpy as np
import math
import collections
# from isoweek import Week
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
import numpy as np

os.getcwd()

'/home/john/network/cnn/deepinsight/dataset_1401'

## data pro-process

In [3]:
sequences_file = './1404.sequences.aln.fasta'
metadata_file = './1404_lineage_report and metadata 20220316.csv'
referenceFile = './onehot-reference.fasta'

In [4]:
metadata = pd.read_csv(metadata_file,encoding='ISO-8859-1')

# split data
nonChange = metadata.loc[metadata['diff']=='N']
trainN,testN=train_test_split(nonChange,test_size= 0.25, random_state=42)

Change = metadata.loc[metadata['diff']=='Y']
trainY,testY=train_test_split(Change,test_size= 0.25, random_state=42)

traindata = trainN.append(trainY)
testdata = testN.append(testY)

print(traindata.shape, testdata.shape)
traindata.to_csv('y_train.csv',index=False)
testdata.to_csv('y_test.csv',index=False)
# np.save('y_train',traindata['diff'])
# np.save('y_test',testdata['diff'])

(1053, 13) (351, 13)


In [5]:
def findReferenceSeq():
	with open(referenceFile) as f:
		currentSeq = ""

		for line in f:
			if ">" not in line:
				currentSeq = currentSeq + line.strip()

	f.close()
	return currentSeq

referenceSeq = findReferenceSeq()

FileNotFoundError: [Errno 2] No such file or directory: './onehot-reference.fasta'

In [7]:
# function for handling weird sequence characters
def clean(x, loc):
	x = x.upper() 
	
	if x == 'T' or x == 'A' or x == 'G' or x == 'C' or x == '-' or x == 'N':
		return x

	if x == 'U' or x == 'Y':
		return 'T'
	
	if x == 'K' or x == 'S':
		return 'G'

	if x == 'M' or x == 'R' or x == 'W' or x == 'H' or x=='V' or x=='D':
		return 'A'

	if x== 'B':
		return 'C'
	
	# otherwise return value from reference
	return referenceSeq[loc]

In [8]:
def getDataLine(seqId, seq):
	dataLine = []
	dataLine.append(seqId)

	newSeq = ""

	# for each character in the sequence
	for index in range(len(seq)):
		newSeq = newSeq + clean(seq[index], index)

	dataLine.append(seq)
	
	return dataLine

In [9]:
# data storage
dataList = []
# dict for lookup efficiency
indiciesToKeep = dict()
idToLineage = dict()

seq_char_dict = {rec.id : rec.seq.upper() for rec in SeqIO.parse(sequences_file, "fasta")}

def readInAndFormatData():
    
	# create a dictionary of sequence ids to their assigned lineages utf-8
	with open(metadata_file, 'r', newline='', encoding='ISO-8859-1') as f:

		for line in f:
			line = line.strip()
			split = line.split(",")
			idToLineage[split[0]] = split[1]

	# close the file
	f.close()

	print("files read in, now processing")

	for key in seq_char_dict.keys():
			if key in idToLineage:
				dataList.append(getDataLine(key, seq_char_dict[key]))
			else:
				print("unable to find the lineage classification for: " + key)

	print('N=',len(dataList))
	return dataList
	
dataList = readInAndFormatData()

files read in, now processing
N= 1404


In [10]:
for index in range(len(dataList[0][1])):
    indiciesToKeep[index] = True
    
print(len(indiciesToKeep))

29903


In [11]:
# remove columns from the data list which don't have any SNPs. 
# We do this because these columns won't be relevant for a logistic regression 
# which is trying to use differences between sequences to assign lineages
def removeOtherIndices(indiciesToKeep):

	# instantiate the final list
	finalList = []

	indicies = list(indiciesToKeep.keys())
	indicies.sort()

	# while the dataList isn't empty
	while len(dataList) > 0:

		# pop the first line (remove)
		line = dataList.pop(0) #reference data
		seqId = line.pop(0) # reference sequence ID

		line = line[0] #remove sequence name ID
		# initialize the finalLine
		finalLine = []

		for index in indicies:
			if index == 0:
				# if its the first index, then that's the lineage assignment, so keep it
				finalLine.append(seqId)
				finalLine.append(line[0])  #0519              
			else:
				# otherwise keep everything at the indices in indiciesToKeep
				finalLine.append(line[index])

                    
		# save the finalLine to the finalList
		finalList.append(finalLine)

	# return
	return finalList

dataList = removeOtherIndices(indiciesToKeep)

In [12]:
# feature
headers = list(indiciesToKeep.keys())
headers[0] = "taxon"
headers.append(29903) # total length:29903

In [13]:
pima = pd.DataFrame(dataList, columns=headers)
pima.head()

,taxon,1,2,3,4,5,6,7,8,9,...,29894,29895,29896,29897,29898,29899,29900,29901,29902,29903
0,hCoV-19/Botswana/R21B67_BHP_AAB78049/2021,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
1,hCoV-19/Yunnan/YN-24/2021,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
2,hCoV-19/Yunnan/YN-39/2021,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
3,hCoV-19/Australia/QLD2012/2021,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
4,hCoV-19/Australia/QLD2027/2021,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N


In [14]:
# nucleotide symbols which can appear
# categories = ['A', 'C', 'G', 'T', '-']
categories = ['A', 'C', 'G', 'T', 'N']

# one hot encoding of all headers other than the first which is the lineage
dummyHeaders = headers[1:]
print(len(dummyHeaders))

29903


In [15]:
# add extra rows to ensure all of the categories are represented, as otherwise 
# not enough columns will be created when we call get_dummies
for i in categories:
	line = [i] * len(dataList[0])
	pima.loc[len(pima)] = line

# get one-hot encoding
pima = pd.get_dummies(pima, columns=dummyHeaders)

# get rid of the fake data we just added
pima.drop(pima.tail(len(categories)).index, inplace=True)

'taxon'

In [ ]:
feature_cols = list(pima)
print(len(feature_cols))
feature_cols.pop(0) # remove column

In [16]:
# fliter X_train and X_test
train = pima.merge(traindata, how='inner', on='taxon')
test = pima.merge(testdata, how='inner', on='taxon')
train.to_csv (r'X_train.csv', index = False)
test.to_csv (r'X_test.csv', index = False)

X_train = train[feature_cols]
X_test = test[feature_cols]
X_train.to_csv (r'X_train.csv', index = False)
X_test.to_csv (r'X_test.csv', index = False)

# np.save('X_train',X_train)
# np.save('X_test',X_test)

## Start to training

In [42]:
# load data
X_train,X_test = np.load('X_train.npy', allow_pickle=True),np.load('X_test.npy', allow_pickle=True)
y_train,y_test = np.load('y_train.npy', allow_pickle=True),np.load('y_test.npy', allow_pickle=True)
# X_train,X_test = pd.read_csv('X_train.csv'),pd.read_csv('X_test.csv')
# y_train,y_test = pd.read_csv('y_train.csv'),pd.read_csv('y_test.csv')

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)
print(len(feature_cols))

(1053, 178334) (351, 178334)
(1053,) (351,)
178334


In [33]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV,validation_curve,KFold
import matplotlib.pyplot as plt
from sklearn import tree
from datetime import datetime
import joblib
import os
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn import metrics
from sklearn.metrics import precision_score,recall_score,accuracy_score,f1_score,roc_auc_score, roc_curve

seed = 42

In [19]:
model_default= DecisionTreeClassifier(criterion='gini', 
    splitter='best', 
    max_depth=None, 
    min_samples_split=2, 
    min_samples_leaf=1, 
    min_weight_fraction_leaf=0.0, 
    max_features=None,  
    random_state= seed, 
    max_leaf_nodes=None, 
    min_impurity_decrease=0.0,  
    class_weight=None,
    ccp_alpha=0.0)


#Fit the algorithm on the data
model_default.fit(X_train,y_train)

#Predict training set:
train_pred = model_default.predict(X_train)
train_predprob = model_default.predict_proba(X_train)

#Prediction on tetsing data
test_pred = model_default.predict(X_test)
test_predprob = model_default.predict_proba(X_test)


In [ ]:
print(metrics.accuracy_score(y_train, train_pred))
print(metrics.accuracy_score(y_test, test_pred))

In [ ]:
print("\nModel Report")
print("Train Accuracy : %.4g" % model_default.score(X_train, y_train))
print("Test Accuracy : %.4g" % model_default.score(X_test, y_test))
# print("AUC Score (Train): %f" % roc_auc_score(y_train, train_predprob))

In [45]:
param_test1 = {"max_depth":range(10,20),"min_samples_split":range(1,10),"min_samples_leaf":range(1,5)}
gsearch1 = GridSearchCV(estimator=DecisionTreeClassifier(criterion='gini',  
                            splitter='best', 
                            max_depth=None, 
                            min_samples_split=2, 
                            min_samples_leaf=1, 
                            min_weight_fraction_leaf=0.0,
                            max_features=None,  
                            random_state=seed, 
                            max_leaf_nodes=None, 
                            min_impurity_decrease=0.0,  
                            class_weight=None,
                            ccp_alpha=0.0),
                   param_grid=param_test1,
                   cv=10,
                   n_jobs=1,
                   return_train_score= True)

gsearch1_result = gsearch1.fit(X_train,y_train)
print("Best: %f using %s" % (gsearch1_result.best_score_, gsearch1_result.best_params_))

In [ ]:
kfolder = KFold(n_splits=10,random_state=seed,shuffle = True) #k-folder
for idx,(train_index, val_index) in enumerate(kfolder.split(X_train, y_train)):
    
    if istime == True:
        model_out = os.path.join(os.getcwd(),f"model save//4067.{j}Model_class{classgroup}_default_time.joblib")
    else:
        model_out = os.path.join(os.getcwd(),f"model save//4067.{j}Model_class{classgroup}_default.joblib")

    X_train_, X_val_ = X_train[train_index], X_train[val_index]
    y_train_, y_val_ = y_train[train_index], y_train[val_index] 

    if idx == 0 :
        model_default.fit(X_train_,y_train_)

        train_accuracy = model_default.score(X_train_, y_train_)
        validate_accuracy = model_default.score(X_val_, y_val_)
        y_pred = model_default.predict(X_test)
        test_accuracy = model_default.score(X_test, y_test)
        joblib.dump(model_default,  model_out, compress=9)
        index = idx+1

    else:
        model_default.fit(X_train_,y_train_)
        validate_ = model_default.score(X_val_, y_val_)

        if validate_accuracy < validate_:

            validate_accuracy = validate_
            train_accuracy = model_default.score(X_train_, y_train_)
            y_pred = model_default.predict(X_test)
            test_accuracy = model_default.score(X_test, y_test)
            joblib.dump(model_default,  model_out, compress=9)
            index = idx+1

    print(f"Cross validaion {index}")
    print("Validation Accuracy:",'%.3f' % validate_accuracy)